In [ ]:
from airo_drake.ur3e_cart import MakeUR3eCartStation
from airo_drake.planners import TowelFoldPlanner
from pydrake.geometry import StartMeshcat, MeshcatVisualizer
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.all import Simulator

In [ ]:
meshcat = StartMeshcat()

In [ ]:
station = MakeUR3eCartStation()
plant = station.GetSubsystemByName("plant")
planner = TowelFoldPlanner(plant, meshcat)

builder = DiagramBuilder()
builder.AddSystem(station)
builder.AddSystem(planner)

# Connect state estimation to planner.
builder.Connect(station.GetOutputPort("ur3e_left_X_WE_estimated"), planner.GetInputPort("left_X_WE_current"))
builder.Connect(station.GetOutputPort("ur3e_right_X_WE_estimated"), planner.GetInputPort("right_X_WE_current"))
builder.Connect(station.GetOutputPort("wsg_left_state_measured"), planner.GetInputPort("left_gripper_state"))
builder.Connect(station.GetOutputPort("wsg_right_state_measured"), planner.GetInputPort("right_gripper_state"))

# Connect planner output to station.
builder.Connect(planner.GetOutputPort("left_X_WE_desired"), station.GetInputPort("ur3e_left_X_WT"))
builder.Connect(planner.GetOutputPort("right_X_WE_desired"), station.GetInputPort("ur3e_right_X_WT"))
builder.Connect(
    planner.GetOutputPort("left_gripper_position_desired"), station.GetInputPort("wsg_left_position")
)
builder.Connect(
    planner.GetOutputPort("right_gripper_position_desired"), station.GetInputPort("wsg_right_position")
)

visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)

diagram = builder.Build()

root_context = diagram.CreateDefaultContext()
planner.root_context = root_context

In [ ]:
simulator = Simulator(diagram)

visualizer.StartRecording(False)
simulator.AdvanceTo(6.0)
visualizer.PublishRecording()